In [1]:
import sys
sys.path.append("../src/") 

import model_manipulation as mm


import cobra
import libsbml
import riptide
from cobra import Model, Reaction, Metabolite
import csv
import pandas as pd
import copy

In [13]:
model = cobra.io.read_sbml_model("../model/ios2164_orig.xml")
rnaseq = pd.read_csv('../data/bs_abundance.csv')



/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/rtsantos3/.local/lib/python3.8/site-packages/optlang/interface.py:1474: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  self._add_constraints(add_constr)


In [80]:
bscell_exp = riptide.read_transcription_file("../data/m_abundance.csv" ,sep = ",", header = True)

In [81]:
model.reactions.RBPOs

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Reaction identifier,RBPOs
Name,Ribulose-biphosphate Oxygenase
Memory address,0x07f31aadd0610
Stoichiometry,"h2o_s + o2_s + rb15bp_s --> 2pglyc_s + 3pg_s + h_s H2O + O2 + D-Ribulose 1,5-bisphosphate --> 2-Phosphoglycolate + 3-Phospho-D-glycerate + H+"
GPR,LOC_Os01g58020 and LOC_Os05g35330 and LOC_Os11g32770 and LOC_Os12g10580 and LOC_Os10g21280 and...
Lower bound,0.0
Upper bound,1000.0


In [58]:
photo_medium = mm.read_medium_csv('../misc/photo_medium.csv', model)

#Define model constraints for photo medium
photo_medium['EX_photonVis(e)'] = 1000
photo_medium['EX_co2(e)'] = 30
photo_medium['EX_o2(e)'] = 10
model.medium = photo_medium

model.medium

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'EX_co2(e)': 30,
 'EX_h2o(e)': 1000000,
 'EX_no3(e)': 1000000,
 'EX_o2(e)': 10,
 'EX_pi(e)': 1000000,
 'EX_so4(e)': 1000000,
 'EX_so3(e)': 1000000,
 'EX_h2s(e)': 1000000,
 'EX_nh4(e)': 1000000,
 'EX_fe2(e)': 1000000,
 'EX_fe3(e)': 1000000,
 'EX_mg2(e)': 1000000,
 'EX_photonVis(e)': 1000}

In [77]:
bscell_rip = riptide.contextualize(model, transcriptome = bscell_exp, samples= 1000,  threshold=1e-9)

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TypeError: contextualize() got an unexpected keyword argument 'rarefy'

In [64]:
sample_fluxes = cobra.flux_analysis.pfba(bscell_rip.model)
sample_fluxes

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/rtsantos3/.local/lib/python3.8/site-packages/optlang/interface.py:1479: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  self._add_constraints(add_constr_sloppy, sloppy=True)


,fluxes,reduced_costs
Straw_Biomass,2.755236,-2.000000
ACCOAC_OSAr,0.278174,-2.000000
FAC_Osr,-0.007852,2.000000
EX_co2(e),-30.000000,2074.799423
EX_h2o(e),95.953927,-2.000000
...,...,...
O2tu,46.592956,-2.000000
H2Otu,-93.185911,2.000000
DM_ADE(s),2.331636,-2.000000
DM_COA(c),0.000028,-2.000000


In [65]:
bs_model = bscell_rip.model

for exchs in bs_model.reactions:
    try:
        print(exchs.id, sample_fluxes[exchs.id])
        
    except KeyError:
        pass


Straw_Biomass 2.755236059401651
ACCOAC_OSAr 0.27817414302930954
FAC_Osr -0.007852422769294706
EX_co2(e) -30.0
EX_h2o(e) 95.95392735763204
EX_no3(e) -13.424230494182503
EX_o2(e) -10.0
EX_pi(e) -0.5685787789261987
EX_so4(e) -0.030032073047478
EX_photonVis(e) -1000.0
PRISM_blue_LED 1000.0
RPSI_410(u) 0.9290386222
RPSI_430(u) 370.517176
RPSI_450(u) 370.18990945177393
RPSII_450(u) 150.21149854822616
RPSII_470(u) 94.361569
RPSII_490(u) 4.578975509999999
CO2tex 30.0
H2Otex -95.95392735763204
NO3tex 13.424230494182503
O2tex 10.0
Pitex 0.5685787789261987
SO4tex 0.030032073047478
PPASc 23.05774650880794
PPASs 3.7377532381842795
PPAIcvc 0.0
APLs 0.1264653351265358
APLm 0.0
CATm 0.0
GLUSfs 0.0
RBPCs 154.09033899036478
CYOO6m 53.19530438887379
NO3Rm 13.592299893806004
ATPSs 311.69860045727853
ATPSm 218.80617332630712
GLUSs 8.181770023553181
GTHRc 0.0
GTHRs 0.0
GTHRm 0.0
ADK1c 6.320728927111833
ADK1s 0.157048455385846
TMDSc 0.002479712453461486
CTPRc 0.0
ATPRc 0.0
GTPRc 0.0
UPRTs 0.23309297062532489

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [67]:
bs_model.reactions.RBPCs

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Reaction identifier,RBPCs
Name,Ribulose-bisphosphate carboxylase
Memory address,0x07f318ba1e3d0
Stoichiometry,"co2_s + h2o_s + rb15bp_s --> 2.0 3pg_s + h_s CO2 + H2O + D-Ribulose 1,5-bisphosphate --> 2.0 3-Phospho-D-glycerate + H+"
GPR,LOC_Os01g58020 and LOC_Os05g35330 and LOC_Os11g32770 and LOC_Os12g10580 and LOC_Os10g21280 and...
Lower bound,120.47430816393823
Upper bound,154.0903389903655


In [76]:
bscell_fva = bscell_rip.flux_variability

bscell_fva

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,minimum,maximum
Straw_Biomass,2.204189,2.755236
ACCOAC_OSAr,0.222539,0.278174
FAC_Osr,-0.007852,-0.006282
EX_co2(e),-30.000000,-15.515959
EX_h2o(e),42.379917,101.576201
...,...,...
O2tu,34.446671,65.868444
H2Otu,-131.736889,-68.893342
DM_ADE(s),0.000000,4.086222
DM_COA(c),0.000022,0.000028


In [69]:
bs_model.reactions.RBPCs.summary()

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/rtsantos3/.local/lib/python3.8/site-packages/optlang/interface.py:1479: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  self._add_constraints(add_constr_sloppy, sloppy=True)


In [70]:
bs_model.reactions.PPCc.summary()

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/rtsantos3/.local/lib/python3.8/site-packages/optlang/interface.py:1479: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  self._add_constraints(add_constr_sloppy, sloppy=True)


Benchmarking Results for BS cells:

Photo medium used:
{'EX_co2(e)': 1000,
 'EX_h2o(e)': 1000000,
 'EX_no3(e)': 1000000,
 'EX_o2(e)': 1000,
 'EX_pi(e)': 1000000,
 'EX_so4(e)': 1000000,
 'EX_so3(e)': 1000000,
 'EX_h2s(e)': 1000000,
 'EX_nh4(e)': 1000000,
 'EX_fe2(e)': 1000000,
 'EX_fe3(e)': 1000000,
 'EX_mg2(e)': 1000000,
 'EX_photonVis(e)': 1500}

No restrictions on CO2 and O2 use

Results:
Rubisco:
Rubisco is active as much as M cells, with roughly similar scale in terms of bounds:
M cell Bounds: 407.9687330150153, 509.9609162687552
BS cell Bounds: 432.16695000671376, 540.2086875083758

However it is 5-6x higher than the normal wt range of 84 umol m-2 s-1 (Ermakova et al. 2021)

PEPC:
Small amounts of PEPC can be seen in both BS and M Cells:
M Cells: (11.43005392808584, 14.287567410107727)
BS Cells: 11.43005392808584, 14.287567410107727

This is overestimated from the 2.40 +- 0.23 range for the experimental measurement




In [82]:
#Let's try the maxfit function

bs_maxfit = riptide.maxfit_contextualize(model=model, transcriptome=bscell_exp)


Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Read LP format model from file /tmp/tmpyvg336hi.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros


/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Read LP format model from file /tmp/tmpse7gpr8p.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros


/home/rtsantos3/.local/lib/python3.8/site-packages/optlang/interface.py:1474: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  self._add_constraints(add_constr)


Read LP format model from file /tmp/tmpjqit0p25.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros
Read LP format model from file /tmp/tmpg3c36vfe.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros
Read LP format model from file /tmp/tmpy5womirw.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros
Read LP format model from file /tmp/tmptmt5hchb.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros
Read LP format model from file /tmp/tmplb3bz2rd.lp
Reading time = 0.00 seconds
: 464 rows, 894 columns, 3834 nonzeros


...

Fraction = 0.9 | Rho = 0.0274 ; p = 0.5563
Testing local objective fractions to 0.35...
Read LP format model from file /tmp/tmprp7r42iy.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros
Read LP format model from file /tmp/tmp_dybfu55.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros
Read LP format model from file /tmp/tmptbgl1mb8.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros
Read LP format model from file /tmp/tmpn5bis0o7.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros
Read LP format model from file /tmp/tmpeddxvn7i.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros
Read LP format model from file /tmp/tmpxhn3_fdk.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros
Read LP format model from file /tmp/tmp8kn_mog9.lp
Reading time = 0.00 seconds
: 464 rows, 890 columns, 3820 nonzeros


/home/rtsantos3/.local/lib/python3.8/site-packages/optlang/interface.py:1474: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  self._add_constraints(add_constr)
/home/rtsantos3/.local/lib/python3.8/site-packages/optlang/interface.py:1474: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  self._add_constraints(add_constr)


Fraction = 0.325 | Rho = 0.1678 ; p = 0.0004
Read LP format model from file /tmp/tmp16m3w14s.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros
Read LP format model from file /tmp/tmpz47411q0.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros
Read LP format model from file /tmp/tmp4_0k7v6q.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros
Read LP format model from file /tmp/tmpcjt3aohc.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros
Read LP format model from file /tmp/tmpuq_0l659.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros
Read LP format model from file /tmp/tmpq7c9dbd5.lp
Reading time = 0.01 seconds
: 1999 rows, 4890 columns, 20956 nonzeros
Read LP format model from file /tmp/tmphsb47avu.lp
Reading time = 0.00 seconds
: 480 rows, 942 columns, 3982 nonzeros


/home/rtsantos3/.local/lib/python3.8/site-packages/optlang/interface.py:1474: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  self._add_constraints(add_constr)
/home/rtsantos3/.local/lib/python3.8/site-packages/optlang/interface.py:1474: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  self._add_constraints(add_constr)


Fraction = 0.375 | Rho = 0.1304 ; p = 0.0046

Context-specific metabolism fit with 0.35 of optimal objective flux

Reactions pruned to 447 from 2445 (81.72% change)
Metabolites pruned to 464 from 1999 (76.79% change)
Flux through the objective DECREASED to ~1.27 from ~3.09 (58.9% change)
Context-specific metabolism correlates with transcriptome (r=0.19, p<0.001 *)

Maxfit RIPTiDe completed in, 8 minutes and 25 seconds 



In [88]:
bsmf_model = bs_maxfit.model

sample_fluxes = cobra.flux_analysis.pfba(bsmf_model)
sample_fluxes

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/rtsantos3/.local/lib/python3.8/site-packages/optlang/interface.py:1479: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  self._add_constraints(add_constr_sloppy, sloppy=True)


,fluxes,reduced_costs
Straw_Biomass,1.266299,-2.0
ACCOAC_OSAr,0.127848,-2.0
FAC_Osr,-0.003609,2.0
EX_co2(e),-30.000000,2.0
EX_h2o(e),59.883316,-2.0
...,...,...
INOSTtr,0.000519,-2.0
O2tu,33.026998,-2.0
H2Otu,-66.053997,2.0
DM_COA(c),0.000013,-2.0


In [ ]:
#Interestingly, using the Maxfit function while constraining both CO2 and O2 intake actually produces 

In [103]:
for rxns in bsmf_model.reactions:
    print(rxns.id, sample_fluxes[rxns.id])

Straw_Biomass 1.266299388083665
ACCOAC_OSAr 0.12784811881970298
FAC_Osr -0.0036089532560384455
EX_co2(e) -30.0
EX_h2o(e) 59.883315845956304
EX_o2(e) -5.55768389063039
EX_pi(e) -0.2613173406230892
EX_h2s(e) -0.013802663330111958
EX_nh4(e) -0.8116725817738547
EX_photonVis(e) -618.5805229425507
PRISM_blue_LED 618.5805229425507
RPSI_410(u) 0.5746851967543029
RPSI_430(u) 229.1947084892771
RPSI_450(u) 207.73205865655373
RPSII_450(u) 114.17811644412595
RPSII_470(u) 58.37022869769959
RPSII_490(u) 2.832465065516933
CO2tex 30.0
H2Otex -59.883315845956304
O2tex 5.55768389063039
Pitex 0.2613173406230892
H2Stex 0.013802663330111958
NH4tex 0.8116725817738547
PPASc 9.783861641978186
PPASs 1.7178617498743003
PPAIcvc 0.0
APLs 0.05812314191304022
APLm 0.0
CATv 0.0
CATm 0.5522412026242403
GLUSfs 1.3382631823084532
RBPCs 69.21210171231309
CYOO6m 0.0
AOXm 36.45335111245866
ATPSs 207.52988224556083
ATPSm 54.58849854891731
GTHRm 0.16227626658292327
ADK1c 2.0913119291640547
ADK1s 0.07217906512077032
URIKGs 0.

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [113]:
bsmf_model.metabolites.rb15bp_s.summary()

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/rtsantos3/.local/lib/python3.8/site-packages/optlang/interface.py:1479: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  self._add_constraints(add_constr_sloppy, sloppy=True)


Percent,Flux,Reaction,Definition
100.00%,69.21,PRUKs,atp_s + ru5p-D_s --> adp_s + h_s + rb15bp_s
Percent,Flux,Reaction,Definition
100.00%,-69.21,RBPCs,co2_s + h2o_s + rb15bp_s --> 2.0 3pg_s + h_s


/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Reaction identifier,O2tex
Name,O2 transport in via diffusion (extracellular to cytosol)
Memory address,0x07f31848e3580
Stoichiometry,o2_e <=> o2_c O2 <=> O2
GPR,
Lower bound,-25.11011557320444
Upper bound,5.557683890551857
